In [640]:
import pandas as pd

file_path = r'C:\Users\lucas\Downloads\FYM\IWSC MRRA Race 23.csv'
data = pd.read_csv(file_path)

data.head()

,Car,Class,Driver,Lap,Lap Time,Session Time,Flag,Location,S01,S02,S03
0,1,GTP,Sebastien Bourdais,1,1:14.788,1:14.788,Green,Track,9.272,39.907,25.454
1,10,GTP,Louis Deletraz,1,1:16.510,1:16.510,Green,Track,9.746,41.452,25.311
2,60,GTP,Tom Blomqvist,1,1:16.768,1:16.768,Green,Track,9.811,41.265,25.217
3,24,GTP,Augusto Farfus,1,1:17.446,1:17.446,Green,Track,9.792,41.798,25.533
4,6,GTP,Nick Tandy,1,1:17.786,1:17.786,Green,Track,9.832,41.885,25.445


In [641]:
from datetime import timedelta
def time_to_timedelta(time_str):
    try:
        parts = list(map(float, time_str.split(':')))
        
        if len(parts) == 2:
            minutes, seconds = parts
            hours = 0
        elif len(parts) == 3:
            hours, minutes, seconds = parts
        else:
            raise ValueError("Incorrect time format")
        
        return seconds + minutes * 60 + hours * 3600
    except ValueError as e:
        print(f"Error converting time: {e}")
        return pd.NaT 
data['Lap Time'] = data['Lap Time'].apply(time_to_timedelta)
data['Session Time'] = data['Session Time'].apply(time_to_timedelta)

data.tail()

,Car,Class,Driver,Lap,Lap Time,Session Time,Flag,Location,S01,S02,S03
17497,57,GTD,Philip Ellis,370,151.190,36138.623,Finish,Track,22.292,72.573,56.325
17498,61,GTDPRO,James Calado,370,149.743,36138.921,Finish,Track,21.411,72.462,55.870
17499,47,GTD,Antonio Fuoco,364,148.715,36139.606,Finish,Track,21.905,72.576,54.234
17500,23,GTD,Alessio Rovera,370,148.675,36140.088,Finish,Track,21.837,72.641,54.197
17501,93,GTD,Kyle Marcelli,355,148.994,36141.042,Finish,Track,23.297,71.424,54.273


In [642]:
gtd_gtd_pro_data = data[data['Class'].isin(['GTD', 'GTD Pro'])]

gtd_gtd_pro_data.head()

,Car,Class,Driver,Lap,Lap Time,Session Time,Flag,Location,S01,S02,S03
23,27,GTD,Ian James,1,114.120,114.120,Green,Track,10.632,44.303,28.732
25,83,GTD,Doriane Pin,1,114.733,114.733,Green,Track,10.811,44.356,28.796
26,12,GTD,Aaron Telitz,1,114.892,114.892,Green,Track,10.823,44.626,28.599
28,97,GTD,Bill Auberlen,1,116.056,116.056,Green,Track,10.753,45.487,28.665
31,57,GTD,Philip Ellis,1,117.583,117.583,Green,Track,11.114,46.124,28.871


In [643]:
# Add a column to record positions
gtd_gtd_pro_data['position'] = 0

# Function to update positions for each lap
def update_positions(df):
    for lap in df['Lap'].unique():
        lap_data = df[df['Lap'] == lap]
        for car_class in lap_data['Class'].unique():
            class_data = lap_data[lap_data['Class'] == car_class]
            class_data = class_data.sort_values(by='Session Time').reset_index(drop=True)
            for idx in range(len(class_data)):
                df.loc[(df['Car'] == class_data.loc[idx, 'Car']) & (df['Lap'] == lap), 'position'] = idx + 1
    return df

# Update the positions in the dataset
gtd_gtd_pro_data = update_positions(gtd_gtd_pro_data)
gtd_gtd_pro_data.head()

C:\Users\lucas\AppData\Local\Temp\ipykernel_28080\2619816431.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gtd_gtd_pro_data['position'] = 0


,Car,Class,Driver,Lap,Lap Time,Session Time,Flag,Location,S01,S02,S03,position
23,27,GTD,Ian James,1,114.120,114.120,Green,Track,10.632,44.303,28.732,1
25,83,GTD,Doriane Pin,1,114.733,114.733,Green,Track,10.811,44.356,28.796,2
26,12,GTD,Aaron Telitz,1,114.892,114.892,Green,Track,10.823,44.626,28.599,3
28,97,GTD,Bill Auberlen,1,116.056,116.056,Green,Track,10.753,45.487,28.665,4
31,57,GTD,Philip Ellis,1,117.583,117.583,Green,Track,11.114,46.124,28.871,5


In [644]:
df = gtd_gtd_pro_data.sort_values(by=['Lap', 'Session Time'])

# Calculate the gap to the car right in front
df['Gap to Car in Front (s)'] = df.groupby('Lap')['Session Time'].diff().fillna(0)

df.head(100)

,Car,Class,Driver,Lap,Lap Time,Session Time,Flag,Location,S01,S02,S03,position,Gap to Car in Front (s)
23,27,GTD,Ian James,1,114.120,114.120,Green,Track,10.632,44.303,28.732,1,0.000
25,83,GTD,Doriane Pin,1,114.733,114.733,Green,Track,10.811,44.356,28.796,2,0.613
26,12,GTD,Aaron Telitz,1,114.892,114.892,Green,Track,10.823,44.626,28.599,3,0.159
28,97,GTD,Bill Auberlen,1,116.056,116.056,Green,Track,10.753,45.487,28.665,4,1.164
31,57,GTD,Philip Ellis,1,117.583,117.583,Green,Track,11.114,46.124,28.871,5,1.527
...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,27,GTD,Ian James,6,125.426,811.605,Green,Track,22.974,63.678,38.774,1,0.000
275,83,GTD,Doriane Pin,6,124.103,811.913,Green,Track,22.609,63.123,38.371,2,0.308
276,12,GTD,Aaron Telitz,6,120.340,812.212,Green,Track,20.894,61.280,38.166,3,0.299
278,97,GTD,Bill Auberlen,6,119.678,812.888,Green,Track,21.135,60.880,37.663,4,0.676


In [645]:
# Adjusted Lap Calculation
df['Adjusted Lap'] = df.apply(lambda x: 0.5 if x['Flag'] == 'Yellow' else 1, axis=1)

# Initialize the 'Stint Length' column
df['Stint Length'] = 0

# Function to calculate the stint length
def calculate_stint_length(group):
    stint_length = 0
    stint_lengths = []
    for index, row in group.iterrows():
        if row['Location'] == 'Pit':
            stint_length = 0
        elif row['Flag'] == 'Green':
            stint_length += 1
        else:
            stint_length += 0.5
        stint_lengths.append(stint_length)
    return pd.Series(stint_lengths, index=group.index)

# Apply the calculate_stint_length function and ensure lengths match
df['Stint Length'] = df.groupby('Car').apply(calculate_stint_length).reset_index(level=0, drop=True)

# Adjusted Lap cumulative sum
df['Adjusted Lap'] = df.groupby('Car')['Adjusted Lap'].cumsum()

# Display the last few rows of the updated DataFrame
df.tail(100)

C:\Users\lucas\AppData\Local\Temp\ipykernel_28080\274807443.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['Stint Length'] = df.groupby('Car').apply(calculate_stint_length).reset_index(level=0, drop=True)


,Car,Class,Driver,Lap,Lap Time,Session Time,Flag,Location,S01,S02,S03,position,Gap to Car in Front (s),Adjusted Lap,Stint Length
17189,27,GTD,Roman De Angelis,362,80.971,35085.192,Green,Track,9.701,42.985,28.285,6,2.227,319.5,26.0
17191,92,GTD,Julien Andlauer,362,81.242,35085.909,Green,Track,9.656,43.027,28.559,7,0.717,318.0,25.5
17194,32,GTD,Mikael Grenier,362,81.721,35087.524,Green,Track,9.758,42.927,29.036,8,1.615,320.0,26.0
17195,97,GTD,Bill Auberlen,362,81.764,35088.136,Green,Track,9.672,43.664,28.428,9,0.612,319.5,26.0
17197,57,GTD,Philip Ellis,362,81.135,35089.561,Green,Track,9.930,43.093,28.112,10,1.425,321.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17492,32,GTD,Mikael Grenier,370,152.804,36134.688,Finish,Track,22.458,72.718,57.628,6,0.872,325.5,31.0
17493,97,GTD,Bill Auberlen,370,153.021,36135.607,Finish,Track,22.610,72.908,57.503,7,0.919,325.0,31.0
17495,80,GTD,Seb Priaulx,370,152.410,36137.520,Finish,Track,22.670,72.771,56.969,8,1.913,325.0,31.0
17497,57,GTD,Philip Ellis,370,151.190,36138.623,Finish,Track,22.292,72.573,56.325,9,1.103,326.0,17.5


In [646]:
adj_avg_stint_dic = {}
def adj_avg_stint(car):
    car_df = df[df['Car'] == car]
    pits = car_df['Location'].value_counts().get('Pit', 1)
    adj_avg_stint_dic[f'Car {car}'] = round(car_df['Adjusted Lap'].max()/pits, 1)

for car in df['Car'].unique():
    adj_avg_stint(car)
print(adj_avg_stint_dic)


{'Car 27': 25.0, 'Car 83': 24.7, 'Car 12': 24.4, 'Car 97': 29.5, 'Car 57': 27.2, 'Car 70': 19.5, 'Car 66': 26.3, 'Car 93': 28.2, 'Car 1': 14.0, 'Car 32': 36.2, 'Car 47': 26.4, 'Car 16': 26.9, 'Car 96': 27.1, 'Car 78': 25.0, 'Car 77': 25.0, 'Car 23': 20.2, 'Car 44': 22.0, 'Car 92': 20.2, 'Car 80': 29.5}


In [647]:
total = 0
for car in adj_avg_stint_dic:
    total += adj_avg_stint_dic[car]
avg_stint = total/len(adj_avg_stint_dic)
print(avg_stint)

25.121052631578944


In [648]:
def identify_early_pitstops(data, car_ids, stint_threshold):
    early_pitstops = []
    for car_id in car_ids:
        car_data = data[data['Car'] == car_id]
        pitstops = car_data[car_data['Location'] == 'Pit']
        
        for idx, pitstop in pitstops.iterrows():
            lap = pitstop['Lap']
            previous_lap = lap - 1
            if previous_lap in car_data['Lap'].values:
                stint_length = car_data[car_data['Lap'] == previous_lap]['Stint Length'].values[0]
                gap = car_data[car_data['Lap'] == previous_lap]["Gap to Car in Front (s)"].values[0]
                if 9 < stint_length < stint_threshold:
                    early_pitstops.append({
                        'Car': car_id,
                        'Pit Lap': lap,
                        'Stint Length': stint_length,
                        'Gap': gap
                    })
    return pd.DataFrame(early_pitstops)

early_pits = identify_early_pitstops(df, df['Car'].unique().tolist(), avg_stint + 5)
early_pits.head(100)

,Car,Pit Lap,Stint Length,Gap
0,27,15,10.0,0.000
1,27,68,15.5,5.755
2,27,111,28.5,28.927
3,27,226,27.5,4.624
4,27,329,21.5,2.216
...,...,...,...,...
95,80,15,10.0,0.912
96,80,72,16.5,6.331
97,80,226,27.5,0.437
98,80,298,30.0,4.544


In [649]:
passed = 0
passed_y = 0
np = 0
np_y = 0

car_list = []
position_list = []
lap_list = []
gap_before_pit_list = []
laps_until_front_car_pit_list = []
stint_list = []
front_car_stint_list = []
pass_bin = []

# Iterate over each row in early_pits
for _, row in early_pits.iterrows():
    try:
        car = row['Car']
        lap = row['Pit Lap']
        pre = df[(df['Car'] == car) & (df['Lap'] == lap - 1)]['position'].values[0]
        if pre != 1:
            front_pre = df[(df['position'] == pre - 1) & (df['Lap'] == lap)]['Car'].values[0]
            front_pit = df[(df['Car'] == front_pre) & (df['Location'] == 'Pit') & (df['Lap'] >= lap)]['Lap'].values[0]
            post = df[(df['Car'] == car) & (df['Lap'] == front_pit + 1)]['position'].values[0]
            front_post = df[(df['Car'] == front_pre) & (df['Lap'] == front_pit + 1)]['position'].values[0]
            gap = df[(df['Car'] == car) & (df['Lap'] == lap - 1)]['Gap to Car in Front (s)'].values[0]
            stint = df[(df['Car'] == car) & (df['Lap'] == lap - 1)]['Stint Length'].values[0]
            front_stint = df[(df['Car'] == front_pre) & (df['Lap'] == front_pit - 1)]['Stint Length'].values[0]

            if gap < 10:
                print(f"Car: {int(car)}, Position before early pit: {pre}, Position {front_pit - lap + 1} laps later: {post}, car in front {front_post}, prior gap {gap} Stint: {stint} front car stint: {front_stint}")
                car_list.append(car)
                position_list.append(pre)
                lap_list.append(lap)
                gap_before_pit_list.append(gap)
                laps_until_front_car_pit_list.append(front_pit - lap)
                stint_list.append(stint)
                front_car_stint_list.append(front_stint)
                if post <= front_post:
                    passed += 1
                    if df[(df['Car'] == front_pre) & (df['Lap'] == front_pit + 1)]['Flag'].values[0] == 'Yellow':
                        passed_y += 1
                    pass_bin.append(1)
                else:
                    np += 1
                    if df[(df['Car'] == front_pre) & (df['Lap'] == front_pit + 1)]['Flag'].values[0] == 'Yellow':
                        np_y += 1
                    pass_bin.append(0)
    except:
        pass

# Create a DataFrame from the collected lists
results_df = pd.DataFrame({
    'Year': 2023,
    'Car': car_list,
    'Position': position_list,
    'Lap': lap_list,
    'Gap Before Pit': gap_before_pit_list,
    'Laps Until Front Car Pit': laps_until_front_car_pit_list,
    'Stint': stint_list,
    'Front Car Stint': front_car_stint_list,
    'Passed': pass_bin
})

# Print results and summary
print(f'Net Gain: {passed} No Gain: {np}')
print(f'Net Gain Green: {passed - passed_y} No Gain Green: {np - np_y}')


Car: 27, Position before early pit: 4, Position 5.0 laps later: 1, car in front 14, prior gap 5.7549999999992 Stint: 15.5 front car stint: 17.5
Car: 27, Position before early pit: 10, Position 1.0 laps later: 9, car in front 9, prior gap 4.623999999999796 Stint: 27.5 front car stint: 27.5
Car: 27, Position before early pit: 7, Position 1.0 laps later: 6, car in front 3, prior gap 2.2160000000003492 Stint: 21.5 front car stint: 28.5
Car: 83, Position before early pit: 7, Position 1.0 laps later: 5, car in front 7, prior gap 1.0859999999993306 Stint: 17.5 front car stint: 17.5
Car: 12, Position before early pit: 5, Position 1.0 laps later: 3, car in front 4, prior gap 1.9369999999998981 Stint: 17.5 front car stint: 17.5
Car: 12, Position before early pit: 5, Position 12.0 laps later: 18, car in front 1, prior gap 3.275999999998021 Stint: 23.5 front car stint: 32.0
Car: 97, Position before early pit: 4, Position 36.0 laps later: 6, car in front 3, prior gap 1.1459999999999582 Stint: 10.0 

In [650]:
results_df.head(100)

,Year,Car,Position,Lap,Gap Before Pit,Laps Until Front Car Pit,Stint,Front Car Stint,Passed
0,2023,27.0,4,68.0,5.755,4.0,15.5,17.5,1
1,2023,27.0,10,226.0,4.624,0.0,27.5,27.5,1
2,2023,27.0,7,329.0,2.216,0.0,21.5,28.5,0
3,2023,83.0,7,72.0,1.086,0.0,17.5,17.5,1
4,2023,12.0,5,72.0,1.937,0.0,17.5,17.5,1
...,...,...,...,...,...,...,...,...,...
62,2023,80.0,16,15.0,0.912,0.0,10.0,10.0,0
63,2023,80.0,9,72.0,6.331,0.0,16.5,17.5,0
64,2023,80.0,5,226.0,0.437,0.0,27.5,27.5,1
65,2023,80.0,3,298.0,4.544,7.0,30.0,33.0,0


In [651]:
num = 0
count = 0
for _, row in results_df.iterrows():
    if row['Laps Until Front Car Pit'] == 0:
        num += 1
        if row['Stint'] > row['Front Car Stint'] and row['Passed'] == 0:
            count += 1
print(num, count)

49 1


In [652]:
results_df.to_csv(r'C:\Users\lucas\Downloads\FYM\Compiled Road Atlanta.csv',mode='a', header=False, index=False)